In [1]:
import json
import random

In [2]:
lang = "en"
diff = "medium"
excl = "easy"

vocabs_pool: set[str] = set()
exclusive_pool: set[str] = set()
alphabets: set[str] = set()

with open(rf"vocabs\{lang}\{lang}-{diff}.json") as file:
    json_data = json.load(file)
    for d in json_data:
        vocabs_pool.add(d)
        exclusive_pool.add(d)
        alphabets = alphabets.union(set(d))

if excl is not None:
    with open(rf"vocabs\{lang}\{lang}-{excl}.json") as file:
        json_data = json.load(file)
        for d in json_data:
            exclusive_pool.remove(d)

num_vocabs = len(vocabs_pool)
print(len(alphabets), "".join(sorted(alphabets)))
print(f"{len(vocabs_pool)=}")
print(f"{len(exclusive_pool)=}")

26 abcdefghijklmnopqrstuvwxyz
len(vocabs_pool)=2439
len(exclusive_pool)=879


In [3]:
def change_character(vocab: str) -> set[str]:
    new_vocabs = set()
    for i, c in enumerate(vocab):
        for a in alphabets:
            if a != c:
                new_vocab = vocab[:i] + a + vocab[i + 1 :]
                new_vocabs.add(new_vocab)

    return new_vocabs


print(change_character("abc"))

{'jbc', 'aqc', 'amc', 'arc', 'abp', 'abf', 'aoc', 'ebc', 'kbc', 'abu', 'ibc', 'abi', 'bbc', 'atc', 'rbc', 'abt', 'alc', 'vbc', 'aac', 'abw', 'abn', 'xbc', 'abh', 'obc', 'lbc', 'awc', 'akc', 'abz', 'abd', 'sbc', 'wbc', 'abq', 'abe', 'azc', 'ahc', 'hbc', 'axc', 'adc', 'ajc', 'abm', 'tbc', 'dbc', 'cbc', 'agc', 'qbc', 'aic', 'abv', 'abb', 'abr', 'aba', 'abs', 'asc', 'pbc', 'ubc', 'abx', 'abl', 'apc', 'fbc', 'ayc', 'abk', 'abg', 'aby', 'zbc', 'auc', 'anc', 'aec', 'abo', 'afc', 'avc', 'acc', 'nbc', 'ybc', 'gbc', 'mbc', 'abj'}


In [4]:
def character_difference(from_: str, to: str) -> int:
    count = 0
    for c1, c2 in zip(from_, to):
        if c1 != c2:
            count += 1
    return count

In [5]:
def distance(
    from_: str, to: str, max_distance: int | None = None
) -> tuple[int, list[str]]:
    if len(from_) != len(to):
        return (-1, [])

    vocab_path = {from_: ""}
    queue = [(from_, 0)]
    while queue:
        (cur_vocab, cur_distance) = queue.pop(0)
        if cur_vocab == to:
            path = [cur_vocab]
            while path[0] != from_:
                path.insert(0, vocab_path[path[0]])
            return (cur_distance, path)

        if cur_distance == max_distance:
            continue

        for new_vocab in change_character(cur_vocab):
            if (new_vocab not in vocab_path) and (new_vocab in vocabs_pool):
                vocab_path[new_vocab] = cur_vocab
                queue.append((new_vocab, cur_distance + 1))

    return (-1, [])

In [6]:
print(distance("admin", "admit"))
print(distance("line", "firm"))
print(distance("センセイ", "モンダイ"))
print(distance("caro", "mano"))

(1, ['admin', 'admit'])
(3, ['line', 'fine', 'fire', 'firm'])
(-1, [])
(-1, [])
(-1, [])


In [7]:
# easy: {"min_distance": 3, "max_distance": 4, "strict": False}
from collections import defaultdict

min_distance = 3
max_distance = 4
strict = False

starter = set()
involved = set()
counts = defaultdict(lambda: [0, 0, 0])  # vocab, pair, exclusive

pairs: list[tuple[str, str]] = []

for i, v1 in enumerate(vocabs_pool):
    for j, v2 in enumerate(vocabs_pool):
        dist = distance(v1, v2, max_distance=max_distance)
        if strict:
            difference = character_difference(v1, v2)
            flag = dist[0] > difference and dist[0] >= min_distance
        else:
            flag = dist[0] >= min_distance

        if flag:
            counts[len(v1)][1] += 1
            starter.add(v1)
            involved = involved.union(set(dist[1]))
            # print(dist)

            if any(v in exclusive_pool for v in dist[1]):
                counts[len(v1)][2] += 1
                pairs.append((v1, v2))

    counts[len(v1)][0] += 1
    if (i + 1) % (num_vocabs // 10) == 0:
        print(f'checking "{v1}"| {i+1}/{num_vocabs}')

checking "users"| 243/2439
checking "rock"| 486/2439
checking "drain"| 729/2439
checking "boxes"| 972/2439
checking "oven"| 1215/2439
checking "ranks"| 1458/2439
checking "setup"| 1701/2439
checking "tail"| 1944/2439
checking "crude"| 2187/2439
checking "child"| 2430/2439


In [8]:
total_vocab = 0
total_combinations = 0
total_pairs = 0
total_exclusive = 0

print_table = []
for k, v in counts.items():
    count_vocab = v[0]
    vocab_combinations = count_vocab * (count_vocab - 1)
    count_pair = v[1]
    count_exlusive = v[2]

    cur_pair_rate = count_pair / vocab_combinations
    cur_exclusive_rate = count_exlusive / vocab_combinations
    to_print = [
        f"len {k}",
        f"{count_vocab}",
        f"{count_pair}/{vocab_combinations}",
        f"{cur_pair_rate:.2%}",
        f"{count_exlusive}/{vocab_combinations}",
        f"{cur_exclusive_rate:.2%}",
    ]
    print_table.append(to_print)

    total_vocab += count_vocab
    total_combinations += vocab_combinations
    total_pairs += count_pair
    total_exclusive += count_exlusive

pair_rate = total_pairs / total_combinations
exclusive_rate = total_exclusive / vocab_combinations
to_print = [
    f"total",
    f"{total_vocab}",
    f"{total_pairs}/{total_combinations}",
    f"{pair_rate:.2%}",
    f"{total_exclusive}/{total_combinations}",
    f"{exclusive_rate:.2%}",
]
print_table.append(to_print)

print(f"----- pair rate & exclusive rate -----")
for items in print_table:
    print(
        f"{items[0]:<5}, #{items[1]:5}:"
        f" {items[2]:>13} = {items[3]:<7} |"
        f" {items[4]:>13} = {items[5]:<7}"
    )
print("-" * 15)

starter_rate = len(starter) / num_vocabs
print(f"starter: {len(starter)}/{num_vocabs} = {starter_rate:.2%}")

participation_rate = len(involved) / num_vocabs
print(f"participation: {len(involved)}/{num_vocabs} = {participation_rate:.2%}")

----- pair rate & exclusive rate -----
len 5, #1652 : 12790/2727452 = 0.47%   |  9723/2727452 = 0.36%  
len 4, #787  :  42274/618582 = 6.83%   |  39235/618582 = 6.34%  
total, #2439 : 55064/3346034 = 1.65%   | 48958/3346034 = 7.91%  
---------------
starter: 1306/2439 = 53.55%
participation: 1350/2439 = 55.35%


In [11]:
random.seed(0)
pairs_shuffled = pairs[:10000]
random.shuffle(pairs_shuffled)
with open(rf"vocabs\{lang}\{lang}-pairs-{diff}.json", "w") as f:
    json.dump(pairs_shuffled, f)

for i in range(5):
    pair = pairs_shuffled[i]
    dist = distance(pair[0], pair[1])
    print(dist)

(3, ['carey', 'cares', 'cases', 'bases'])
(3, ['book', 'boot', 'root', 'room'])
(3, ['tired', 'tires', 'tiles', 'miles'])
(4, ['dual', 'deal', 'dell', 'fell', 'feel'])
(4, ['pool', 'poll', 'pill', 'will', 'wall'])


In [10]:
def create_distance_graph():
    distance_graph: dict[tuple[str, str], tuple[int, list[str]]] = {}

    def calculate_distance(from_: str, to: str):
        if (from_, to) in distance_graph:
            return

        min_distance = -1
        min_path = []
        for new_vocab in change_character(from_):
            if new_vocab in vocabs_pool:
                calculate_distance(new_vocab, to)
                distance, path = distance_graph[(new_vocab, to)]
                if min_distance == -1 or 0 <= distance < min_distance:
                    min_distance = distance + 1
                    min_path = [from_] + path
        distance_graph[(from_, to)] = (min_distance, min_path)

    for v1 in vocabs_pool:
        for v2 in vocabs_pool:
            calculate_distance(v1, v2)
    return distance_graph


# distance_graph = create_distance_graph()
# print(distance_graph)